<a href="https://colab.research.google.com/github/mikashaw/SSVEPBrainWaveClassifier/blob/main/CompactCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import scipy.io

In [ ]:
# build tensorflow model from scratch
# use tensorflow  

In [ ]:
!git clone https://github.com/aliasvishnu/EEGNet.git

Cloning into 'EEGNet'...
remote: Enumerating objects: 33, done.
remote: Total 33 (delta 0), reused 0 (delta 0), pack-reused 33
Unpacking objects: 100% (33/33), done.


In [ ]:
!git clone https://github.com/mnakanishi/12JFPM_SSVEP.git

Cloning into '12JFPM_SSVEP'...
remote: Enumerating objects: 43, done.
remote: Total 43 (delta 0), reused 0 (delta 0), pack-reused 43
Unpacking objects: 100% (43/43), done.


In [ ]:
# load in data for one user

data = scipy.io.loadmat("/content/12JFPM_SSVEP/data/s1.mat")

In [ ]:

# preprocess and reshaoe the data

#make a different class if want to split into train and test


class Preprocess:

  """preprocesses data with shape [Number of targets, Number of channels, Number of sampling points, Number of trials] 
  returning x_train and y_train dataset"""

  def __init__(self, data):

    self.data = data

    self._num_targets = None
    self._num_channels = None
    self._num_sampling_points = None
    self._num_trials = None 

  def process(self):
    """
    main public function: 
    Returns: reshaped data to shape [trials, samples, channels] in the form of a numpy array
    """

    # get data in the form of an numpy array
    np_data = self._to_numpy_array(self.data)
    #reshape the array to the desired format
    reshaped_arr = self._reshape_data(np_data)
    # get the training labels from the dataset and strip off labels from the data
    np_data, labels = self._get_labels(reshaped_arr)

    return np_data, labels

  def _to_numpy_array(self, data):

    data= np.array(data['eeg'], dtype='float32')

    #set properties
    self._num_targets = data.shape[0]
    self._num_channels = data.shape[1]
    self._num_sampling_points = data.shape[2]
    self._num_trials = data.shape[3]

    return data

  def _reshape_data(self, data):

    reshaped_data = data.reshape(self._num_targets, self._num_trials,1, self._num_sampling_points, self._num_channels)
    return reshaped_data
    

  def _get_labels(self, data):

    #get a labels for every sample in a trial there are 8 chennels 

    labels = []
    
    for class_num in range(0, self._num_targets):
      labels.append(([class_num]*self._num_trials*self._num_sampling_points))

    data= data[0,:,:,:]
    
    labels = np.array(np.squeeze(labels))

    return data, labels 





In [ ]:
preprocessor = Preprocess(data)

In [ ]:
x_train, y_train = preprocessor.process()

In [ ]:
x_train.shape

(15, 1, 1114, 8)

In [ ]:
# model

import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

In [ ]:

class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 120
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 64), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 120 timepoints. 
        self.fc1 = nn.Linear(4*2*7, 1)
        

    def forward(self, x):
        # Layer 1
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)
        x = x.permute(0, 3, 1, 2)
        
        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.25)
        x = self.pooling2(x)
        
        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)
        
        # FC Layer
        x = x.reshape(-1, 4*2*7)
        x = F.sigmoid(self.fc1(x))
        return x









In [ ]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.3 MB/s eta 0:14:59tcmalloc: large alloc 1147494400 bytes == 0x56472c172000 @  0x7f7354090615 0x5646f20e94cc 0x5646f21c947a 0x5646f20ec2ed 0x5646f21dde1d 0x5646f215fe99 0x5646f215a9ee 0x5646f20edbda 0x5646f215fd00 0x5646f215a9ee 0x5646f20edbda 0x5646f215c737 0x5646f21dec66 0x5646f215bdaf 0x5646f21dec66 0x5646f215bdaf 0x5646f21dec66 0x5646f215bdaf 0x5646f20ee039 0x5646f2131409 0x5646f20ecc52 0x5646f215fc25 0x5646f215a9ee 0x5646f20edbda 0x5646f215c737 0x5646f215a9ee 0x5646f20edbda 0x5646f215b915 0x5646f20edafa 0x5646f215bc0d 0x5646f215a9ee
     |█████████████████               | 1055.7 MB 1.3 MB/s eta 0:12:17tcmalloc: large alloc 1434370048 bytes == 0x5647707c8000 @  0x7f7354090615 0x5646f20e94cc 0x5646f21c947a 0x5646f20ec2ed 0x5646f21dde1d 0x5646f215fe99 0x5646f215a9ee 0x5646f20edbda 0x5646f215fd00 0x5646f215a9ee 0x5646f20edbda 0x5646f215c737 0x5646f21dec66 0x5646f215bd

In [ ]:
net = EEGNet().cuda(0)
print(net.forward(Variable(torch.Tensor(np.random.rand(1, 1, 120, 64)).cuda(0))))
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters())

tensor([[0.1177]], device='cuda:0', grad_fn=<SigmoidBackward0>)


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
def evaluate(model, X, Y, params = ["acc"]):
    results = []
    batch_size = 100
    
    predicted = []
    
    for i in range(len(X)//batch_size):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(torch.from_numpy(X[s:e]).cuda(0))
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
        
        
    inputs = Variable(torch.from_numpy(X).cuda(0))
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
    
    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted))
            recall = recall_score(Y, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results




In [ ]:
# data

X_train = np.random.rand(100, 1, 120, 64).astype('float32') # np.random.rand generates between [0, 1)
y_train = np.round(np.random.rand(100).astype('float32')) # binary data, so we round it to 0 or 1.

X_val = np.random.rand(100, 1, 120, 64).astype('float32')
y_val = np.round(np.random.rand(100).astype('float32'))

X_test = np.random.rand(100, 1, 120, 64).astype('float32')
y_test = np.round(np.random.rand(100).astype('float32'))


In [ ]:
len(X_train)

100

In [ ]:


batch_size = 32

for epoch in range(10):  # loop over the dataset multiple times
    print("\nEpoch: ", epoch)
    
    running_loss = 0.0
    for i in range(len(X_train)//batch_size-1):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(X_train[s:e])
        labels = torch.FloatTensor(np.array([y_train[s:e]]).T*1.0)
        
        # wrap them in Variable
        inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        
        optimizer.step()
        
        running_loss += loss.data
    
    # Validation accuracy
    params = ["acc", "auc", "fmeasure"]
    print(params)
    print("Training Loss ", running_loss)
    print("Train - ", evaluate(net, X_train, y_train, params))
    print("Validation - ", evaluate(net, X_val, y_val, params))
    print("Test - ", evaluate(net, X_test, y_test, params))


Epoch:  0
['acc', 'auc', 'fmeasure']
Training Loss  tensor(1.4564, device='cuda:0')
Train -  [0.56, 0.6033653846153846, 0.4054054054054054]
Validation -  [0.56, 0.5362144857943177, 0.42105263157894735]
Test -  [0.45, 0.45472756410256415, 0.30379746835443033]

Epoch:  1
['acc', 'auc', 'fmeasure']
Training Loss  tensor(1.3946, device='cuda:0')
Train -  [0.58, 0.5272435897435898, 0.5227272727272727]
Validation -  [0.53, 0.5314125650260104, 0.49462365591397855]
Test -  [0.51, 0.4627403846153846, 0.4731182795698925]

Epoch:  2
['acc', 'auc', 'fmeasure']
Training Loss  tensor(1.2690, device='cuda:0')
Train -  [0.52, 0.5220352564102564, 0.48936170212765956]
Validation -  [0.52, 0.482593037214886, 0.5555555555555555]
Test -  [0.53, 0.5516826923076923, 0.5346534653465348]

Epoch:  3
['acc', 'auc', 'fmeasure']
Training Loss  tensor(1.3434, device='cuda:0')
Train -  [0.51, 0.5544871794871795, 0.5050505050505051]
Validation -  [0.54, 0.5530212084833934, 0.5892857142857144]
Test -  [0.5, 0.5332532

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.func

Test -  [0.45, 0.47716346153846156, 0.4444444444444444]

Epoch:  6
['acc', 'auc', 'fmeasure']
Training Loss  tensor(1.2233, device='cuda:0')
Train -  [0.62, 0.6145833333333334, 0.6041666666666666]
Validation -  [0.56, 0.5474189675870349, 0.5416666666666666]
Test -  [0.46, 0.4547275641025641, 0.4807692307692308]

Epoch:  7
['acc', 'auc', 'fmeasure']
Training Loss  tensor(1.3021, device='cuda:0')
Train -  [0.52, 0.5913461538461539, 0.4666666666666667]
Validation -  [0.5, 0.5382152861144458, 0.4565217391304348]
Test -  [0.54, 0.5520833333333334, 0.4772727272727273]

Epoch:  8
['acc', 'auc', 'fmeasure']
Training Loss  tensor(1.1479, device='cuda:0')
Train -  [0.61, 0.6049679487179487, 0.5411764705882353]
Validation -  [0.45, 0.43657462985194084, 0.39560439560439564]
Test -  [0.48, 0.4166666666666667, 0.40909090909090906]

Epoch:  9
['acc', 'auc', 'fmeasure']
Training Loss  tensor(1.2805, device='cuda:0')
Train -  [0.61, 0.6534455128205128, 0.5185185185185185]
Validation -  [0.5, 0.51620648

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.func

In [ ]:
d = [4,5,6,6]